In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import KFold,ShuffleSplit

In [2]:
cd\Users\prave\datasets\csv files\competition\default loan prdiction

C:\Users\prave\datasets\csv files\competition\default loan prdiction


In [3]:
train=pd.read_csv('train_LZV4RXX.csv')
test=pd.read_csv("test_4zJg83n.csv")
sub=pd.read_csv('sample_submission_tbPU9qQ.csv')

In [4]:
# Merging both train and test dataset for EDA
train['test_data']=0
test['test_data']=1
test['loan_default']=np.NaN

df=pd.concat([train,test])

In [5]:
df['education'].fillna(df['education'].median(),inplace=True)

In [6]:
df=pd.get_dummies(data=df,columns=['proof_submitted'],drop_first=True)

# Feature Selection

In [7]:
temp_x_train=df[df['test_data']==0].drop(['loan_default'],axis=1)
temp_y_train=df[df['test_data']==0]['loan_default']

In [9]:
# Splitting the dataset into train set and test set
X_train = df[df['test_data']==0].drop(['loan_default'], axis=1)
X_train.drop(['test_data'], axis=1, inplace=True)
y_train = df[df['test_data']==0]['loan_default']
X_test = df[df['test_data']==1].drop(['loan_default'], axis=1)
X_test.drop(['test_data'], axis=1, inplace=True)

In [10]:
# Feature Scaling using StandardScalar
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
X_train = ss.fit_transform(X_train)
X_test = ss.transform(X_test)

In [11]:
from sklearn.model_selection import cross_val_score,KFold,ShuffleSplit
from sklearn.metrics import f1_score

In [101]:
from lightgbm import LGBMClassifier

lgbm=LGBMClassifier(num_iteration=105,boosting='dart',num_leaves=60,learning_rate=0.15)
lgbm.fit(X_train,y_train)
cv = cross_val_score(lgbm, X_train, y_train, cv=5, scoring='accuracy')
print('--- Average Accuracy: {}% ---'.format(round(cv.mean()*100, 4)))
print("F1 score : {}".format(f1_score(lgbm.predict(X_train),y_train)))

[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart
[LightGBM] [Warning] num_iterations is set=105, num_iteration=105 will be ignored. Current value: num_iterations=105
[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart
[LightGBM] [Warning] num_iterations is set=105, num_iteration=105 will be ignored. Current value: num_iterations=105
[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart
[LightGBM] [Warning] num_iterations is set=105, num_iteration=105 will be ignored. Current value: num_iterations=105
[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart
[LightGBM] [Warning] num_iterations is set=105, num_iteration=105 will be ignored. Current value: num_iterations=105
[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosti

In [103]:
pred=lgbm.predict(X_test)
submission=pd.DataFrame({"loan_id":test['loan_id'],"loan_default":pred})

submission.to_csv('lgbm58582_submission.csv',index=False)